In [22]:
import numpy as np 

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit import Aer
from qiskit.primitives import Sampler, BackendSampler
from qiskit.circuit.library import TwoLocal

from src.Quantum_Optimizer_FGS_modified import Quantum_Optimizer_FGS

In [34]:
target_mdl = QuadraticProgram()
target_mdl.read_from_lp_file('../lp_files/Random_seq8.lp')

N = target_mdl.get_num_vars()
print(N)

#print(target_mdl.prettyprint())

36


In [35]:
cplex_result = CplexOptimizer().solve(target_mdl)
cplex_optval = cplex_result.fval
cplex_xsol = cplex_result.x.astype(int)

print(
        '-'*110,
        '[CPLEX Result]',
        f'   optimal value: {cplex_optval:.3f}',
        f'   optimal solution (binary): {cplex_xsol}',
        f'   status: {cplex_result.status.name}',
        '-'*110,
        sep='\n'
    )

--------------------------------------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: -25765.500
   optimal solution (binary): [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
   status: SUCCESS
--------------------------------------------------------------------------------------------------------------


In [5]:
qubo = QuadraticProgramToQubo().convert(target_mdl)

#print(qubo.prettyprint())

In [6]:
num_qubits = N
print(f'number of system qubits: {num_qubits}')

number of system qubits: 64


In [7]:
reps = 1

ansatz = TwoLocal(
    num_qubits = num_qubits,
    rotation_blocks = 'ry', 
    entanglement_blocks = 'cz',
    entanglement = 'pairwise', 
    reps = reps,
    insert_barriers = True
    )

n_params = ansatz.num_parameters
print(f'number of params: {n_params}')

number of params: 128


In [8]:
shots = 2**10
backend = Aer.get_backend('aer_simulator_matrix_product_state')

sampler = BackendSampler(backend=backend)
sampler.set_options(shots=shots)

print(f'shots = {shots}')

shots = 1024


In [9]:
def objective(x):
    objval = qubo.objective.evaluate(x)
    return objval

In [10]:
print(f'reference value: {cplex_optval:.3f}')
print('-'*80)

QuantumOptimizer = Quantum_Optimizer_FGS()
QuantumOptimizer.set_optimizer(num_qubits, ansatz, sampler)

theta_initial = np.hstack([np.full(num_qubits, np.pi/2), np.zeros(num_qubits*ansatz.reps)])
max_energy =  np.sum([abs(elem) for elem in qubo.objective.quadratic.to_dict().values() if elem >= 0.0])
min_energy =  np.sum([elem for elem in qubo.objective.quadratic.to_dict().values() if elem < 0.0]) 

print(max_energy, min_energy)

#filter_func = {"exp": np.exp(x), "power"=x**(0.7), "log"=np.ln(x)}
result = QuantumOptimizer.run(
    objective = objective, 
    ee_bounds = {'min_ee': min_energy, 'max_ee':max_energy},
    op_trans = lambda x: np.exp(-8*x), 
    qc_params_init = theta_initial,
    max_epoch = 100, 
    random_update = True, 
    intermediate_result_display = True,
    refval = cplex_optval
)

reference value: -118.200
--------------------------------------------------------------------------------
73950.0 -460.0
epoch = 0 	 step = 0 	 expectation = 18175.522
epoch = 0 	 step = 1 	 expectation = 17580.785
epoch = 0 	 step = 2 	 expectation = 17502.637
epoch = 0 	 step = 3 	 expectation = 17166.141
epoch = 0 	 step = 4 	 expectation = 16626.904
epoch = 0 	 step = 5 	 expectation = 16286.131
epoch = 0 	 step = 6 	 expectation = 16046.602
epoch = 0 	 step = 7 	 expectation = 15938.723
epoch = 0 	 step = 8 	 expectation = 15249.705
epoch = 0 	 step = 9 	 expectation = 15054.962
epoch = 0 	 step = 10 	 expectation = 14219.794
epoch = 0 	 step = 11 	 expectation = 14380.356
epoch = 0 	 step = 12 	 expectation = 14612.708
epoch = 0 	 step = 13 	 expectation = 13899.306
epoch = 0 	 step = 14 	 expectation = 13957.240
epoch = 0 	 step = 15 	 expectation = 13845.812
epoch = 0 	 step = 16 	 expectation = 13266.051
epoch = 0 	 step = 17 	 expectation = 13425.230
epoch = 0 	 step = 18 	 

KeyboardInterrupt: 

In [ ]:
x = QuantumOptimizer.best_x
objval = qubo.objective.evaluate(x)

print(
        '-'*110,
        '[Quantum Result]',
        f'   objective:   {objval:.3f}',
        f'   solution:    {x}',
        f'   is feasible: {target_mdl.is_feasible(x)}',
        '-'*110,
        sep='\n'
    )